In [25]:
%reset -f
from IPython import display

In [ ]:
%%HTML
<iframe style="background-color: #fff;" width="1000" height="500" src="http://127.0.0.1:8080">
</iframe>

In [ ]:
%%javascript
let data;
window.addEventListener("message", message_handler);
element.text("")

function message_handler(event){
    event.origin === 'http://127.0.0.1:8080'? data = event.data : ' ';
    element.text(data);
}